In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [15]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [16]:
# Start defining a ConfigurationSpace for the Optimizer to search.
input_space = CS.ConfigurationSpace(seed=1234)

max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=10)
max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=2)
level0_slowdown_writes_trigger=CS.Integer("level0_slowdown_writes_trigger", (1,50), default=20) 
level0_stop_writes_trigger=CS.Integer("level0_stop_writes_trigger", (1,50), default=36) 
target_file_size_base=CS.Integer("target_file_size_base", (1,500 * 1048576), default=64 * 1048576)
target_file_size_multiplier=CS.Integer("target_file_size_multiplier", (1,20), default=1)
max_background_jobs=CS.Integer("max_background_jobs", (1,20), default=2)
max_subcompactions=CS.Integer("max_subcompactions", (1,20), default=1)
avoid_flush_during_shutdown=CS.Categorical("avoid_flush_during_shutdown", [True,False], default=True) 

In [17]:
input_space.add_hyperparameters([max_bytes_for_level_multiplier,max_write_buffer_number,level0_slowdown_writes_trigger,level0_stop_writes_trigger,
                                target_file_size_base,target_file_size_multiplier,max_background_jobs,max_subcompactions,avoid_flush_during_shutdown])

[max_bytes_for_level_multiplier, Type: UniformFloat, Range: [2.0, 20.0], Default: 10.0,
 max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 level0_slowdown_writes_trigger, Type: UniformInteger, Range: [1, 50], Default: 20,
 level0_stop_writes_trigger, Type: UniformInteger, Range: [1, 50], Default: 36,
 target_file_size_base, Type: UniformInteger, Range: [1, 524288000], Default: 67108864,
 target_file_size_multiplier, Type: UniformInteger, Range: [1, 20], Default: 1,
 max_background_jobs, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_subcompactions, Type: UniformInteger, Range: [1, 20], Default: 1,
 avoid_flush_during_shutdown, Type: Categorical, Choices: {True, False}, Default: True]

In [18]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [19]:
optimizer

SmacOptimizer(space_adapter=None)

In [20]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    #print(new_val)
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [21]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY)
            try:
                buffer_size = 128
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [22]:
def run_optimization():
    # get a new config value suggestion to try from the optimizer.
    suggested_value = optimizer.suggest()
    pass_values_to_interface(suggested_value)
    index=read_epochs_pipe()
    time.sleep(5)
    if index==-1:
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    print("suggested value",suggested_value)
    print("target value", target_value)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 3
for i in range(n_iterations):
    run_optimization()

Reported epochs: 4942
suggested value    avoid_flush_during_shutdown  level0_slowdown_writes_trigger  \
0                        False                              15   

   level0_stop_writes_trigger  max_background_jobs  \
0                          46                    5   

   max_bytes_for_level_multiplier  max_subcompactions  \
0                       17.609564                  17   

   max_write_buffer_number  target_file_size_base  target_file_size_multiplier  
0                        7               84427327                           13  
target value 1321.25
Reported epochs: 4952
suggested value    avoid_flush_during_shutdown  level0_slowdown_writes_trigger  \
0                         True                              33   

   level0_stop_writes_trigger  max_background_jobs  \
0                          11                   17   

   max_bytes_for_level_multiplier  max_subcompactions  \
0                        9.922628                   5   

   max_write_buffer_number 

In [23]:
optimizer.get_observations()

,avoid_flush_during_shutdown,level0_slowdown_writes_trigger,level0_stop_writes_trigger,max_background_jobs,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,score
0,False,15,46,5,17.609564,17,7,84427327,13,1321.25
0,True,33,11,17,9.922628,5,19,370845809,1,1272.60
0,True,8,36,7,3.679120,8,3,433598358,9,1231.40


In [24]:
optimizer.get_best_observation()

,avoid_flush_during_shutdown,level0_slowdown_writes_trigger,level0_stop_writes_trigger,max_background_jobs,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,score
0,True,8,36,7,3.67912,8,3,433598358,9,1231.4


In [25]:
optimum_value=optimizer.suggest()
pass_values_to_interface(optimum_value)

In [26]:
print(optimum_value)

   avoid_flush_during_shutdown  level0_slowdown_writes_trigger  \
0                        False                              40   

   level0_stop_writes_trigger  max_background_jobs  \
0                          22                   14   

   max_bytes_for_level_multiplier  max_subcompactions  \
0                       15.028887                  11   

   max_write_buffer_number  target_file_size_base  target_file_size_multiplier  
0                       11              163048777                           20  


In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}